# Face Recognition

In [1]:
import cv2 as cv
import os
import numpy as np

In [2]:
# Defining Scale Function
def scaleFrame(frame,scale=0.75):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions =  (width,height)

    return cv.resize(frame,dimensions,interpolation = cv.INTER_CUBIC)

In [3]:
# Drawing box and writing Name with Confidence
def faceBox(image,x,y,width,height,label,confidence):
    cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)
    cv.putText(image, 'Person {} : {}%'.format(label,round(confidence,2)), (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image

# Training the recognizer

In [5]:
def get_images_and_labels(dataset_path):
    images = []
    labels = []

    for person_dir in os.listdir(dataset_path):
        person_path = dataset_path+"/"+person_dir
        if os.path.isdir(person_path):
            for image_name in os.listdir(person_path):
                image_path = person_path+"/"+image_name
                image = cv.imread(image_path,cv.IMREAD_GRAYSCALE)
                images.append(image)
                labels.append(int(person_dir))  # Use folder name as label

    return images, labels
    
recognizer =  cv.face.LBPHFaceRecognizer_create()
image_paths, labels = get_images_and_labels("C:/Users/mohit/Programming/My_Projects/Attendance System Using Face Recognition/Dataset")
# print(labels)
recognizer.train(image_paths, np.array(labels))

# Face Recognition

In [6]:
# Face Detection Using CascadeClassifier
videoStream = cv.VideoCapture(0)

while True:
    # Capturing Frame from camera
    isRet, frame = videoStream.read()

    # Scaling Image
    frame = scaleFrame(frame,scale=1.35)

    # Converting frame to gray scale
    gray_frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    # Loading face detector from opencv library
    face_detector_frontal = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
    face_detector_profile = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_profileface.xml")

    # Detecting Faces
    face_frontal = face_detector_frontal.detectMultiScale(gray_frame, minNeighbors=10, minSize=(30,30))
    face_profile = face_detector_profile.detectMultiScale(gray_frame, minNeighbors=10, minSize=(30,30))

    # Creating Box Around Face
    if len(face_frontal) > 0:
        for (x, y, w, h) in face_frontal:
            face = gray_frame[y:y+h, x:x+w]
            label,confidence = recognizer.predict(face)
            frame = faceBox(frame,x,y,w,h,label,confidence)
    elif len(face_profile) > 0:
        for (x, y, w, h) in face_profile:
            face = gray_frame[y:y+h, x:x+w]
            label,confidence = recognizer.predict(face)
            frame = faceBox(frame,x,y,w,h,label,confidence)

    # Displaying Image
    cv.imshow("Face Recognition",frame)

    # Checking for keyboard input to 
    if cv.waitKey(20) & 0xFF == ord('q'):
        break


videoStream.release()
cv.destroyAllWindows()